In [41]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import widgets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Get Data from Website

In [2]:
!wget https://s3.amazonaws.com/drivendata/data/57/public/train_values.csv -nc -P ./nepal/
!wget https://s3.amazonaws.com/drivendata/data/57/public/train_labels.csv -nc -P ./nepal/
!wget https://s3.amazonaws.com/drivendata/data/57/public/test_values.csv -nc -P ./nepal/

File ‘./nepal/train_values.csv’ already there; not retrieving.

--2019-11-11 21:23:18--  https://s3.amazonaws.com/drivendata/data/57/public/train_labels.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.237.85
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.237.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2330792 (2.2M) [text/csv]
Saving to: ‘./nepal/train_labels.csv’

train_labels.csv    100%[===================>]   2.22M  --.-KB/s    in 0.06s   

2019-11-11 21:23:18 (38.4 MB/s) - ‘./nepal/train_labels.csv’ saved [2330792/2330792]

--2019-11-11 21:23:18--  https://s3.amazonaws.com/drivendata/data/57/public/test_values.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.146.221
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.146.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7815385 (7.5M) [text/csv]
Saving to: ‘./nepal/test_values.csv’

test_values.csv     100%[===================>]   7.

# Import Data

In [18]:
X = pd.read_csv('./nepal/train_values.csv', index_col='building_id')

In [19]:
X.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 38 columns):
geo_level_1_id                            260601 non-null int64
geo_level_2_id                            260601 non-null int64
geo_level_3_id                            260601 non-null int64
count_floors_pre_eq                       260601 non-null int64
age                                       260601 non-null int64
area_percentage                           260601 non-null int64
height_percentage                         260601 non-null int64
land_surface_condition                    260601 non-null object
foundation_type                           260601 non-null object
roof_type                                 260601 non-null object
ground_floor_type                         260601 non-null object
other_floor_type                          260601 non-null object
position                                  260601 non-null object
plan_configuration                        2606

In [25]:
y = pd.read_csv('nepal/train_labels.csv', index_col='building_id')['damage_grade']

# Attempt 1: Model w/ One Feature

In [29]:
def housing_plot(X, y):
    def plotter(column):
        valid_rows = X[column].notna()
        plt.plot(X.loc[valid_rows, column], y[valid_rows], '.', color='k')
        plt.ylabel('Damage Level')
        plt.yticks([1,2,3])
    
    return plotter

dropdown_values = sorted(X.columns)
widgets.interact(housing_plot(X, y), column=dropdown_values);

interactive(children=(Dropdown(description='column', options=('age', 'area_percentage', 'count_families', 'cou…

In [31]:
X_height = X[['height_percentage']]
X_height.head()

,height_percentage
building_id,
802906,5
28830,7
94947,5
590882,5
201944,9


## Train-test split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_height, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [40]:
one_feat_model = LogisticRegression(solver='lbfgs', multi_class='auto')
one_feat_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
y_train_pred = one_feat_model.predict(X_train)

## Compare in- and out-sample metrics (f1 score)

In [47]:
print('In-sample f1 score:')
f1_score(y_train, y_train_pred, average='micro')

In-sample f1 score:


0.5699779355333845

In [49]:
y_test_pred = one_feat_model.predict(X_test)
print('Out-sample f1 score:')
f1_score(y_test, y_test_pred, average='micro')

Out-sample f1 score:


0.5660290477926364

## Create submission

In [53]:
X_comp_test = pd.read_csv('nepal/test_values.csv', index_col='building_id')
X_comp_test.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0


In [55]:
y_comp_pred = one_feat_model.predict(X_comp_test[['height_percentage']])

In [59]:
y_submission = pd.DataFrame(y_comp_pred, index=X_comp_test.index, columns=['damage_grade'])

In [60]:
y_submission.to_csv('nepal/2019-11-11_submission.csv')

**Score: 0.56**

# Attempt 2: All Numerical Features